In [1]:
#Cas 3:
using Cbc
using JuMP

NB_CLIENTS = 2
NB_MAGASINS = 3
NB_PRODUITS = 2

#le cout du produit Pj dans le magasin Mi 
#C = rand(1:5, NB_MAGASINS, NB_PRODUITS)
C = [1 1; 2 3; 3 2]

#Le nombre disponible en stock de Pj dans le magasin Mi
#S = rand(10:15, NB_MAGASINS, NB_PRODUITS)
S = [2.5 1; 1 2; 2 1]

#Le nombre total de produit Pj demandé par le client k
#q = rand(1:5, NB_CLIENTS, NB_PRODUITS)
Q = [2 0;1 3]

# Le cout d'expedition a partir de magasin Mi pour liverer au client k
#D = rand(0:3, NB_MAGASINS, NB_CLIENTS)
D = [1 0; 0 2; 0 1]

# set optimizer
model3 = Model(Cbc.Optimizer)

# define variables
@variable(model3, A[1:NB_MAGASINS,1:NB_PRODUITS,1:NB_CLIENTS] >= 0, Int)

# define objective function
@objective(model3, Min, sum(sum( sum((C[i,j] + D[i, k]) * A[i,j,k] for i in 1:NB_MAGASINS) for k in 1:NB_CLIENTS) for j in 1:NB_PRODUITS))

# Aucun magasin ne peut livrer plus que ce qu'il a en stock
for i in 1:NB_MAGASINS
    for j in 1:NB_PRODUITS
        @constraint(model3,  sum(A[i,j,k] for k in 1:NB_CLIENTS) <= S[i,j])
    end
end

# Le nombre total demandé par chaque client
for j in 1:NB_PRODUITS
    for k in 1:NB_CLIENTS
        @constraint(model3,  sum(A[i,j,k] for i in 1:NB_MAGASINS ) == Q[k,j])
    end
end

optimize!(model3)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 14 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 6 rows, 9 columns (9 integer (6 of which binary)) and 17 elements
Cbc0012I Integer solution of 14 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0001I Search completed - best objective 14, took 0 iterations and 0 nodes (0.00 seconds)
Cbc0035I Maximum depth 0, 0 variables fixed on reduced cost
Cuts at root node changed objective from 14 to 14
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and created 

In [4]:
# print solution
println("Solution obtenue:")
println("\t benefice = $(objective_value(model3))")
for i in 1:3
    for j in 1:2
        for k in 1:2 
            if (value(A[i,j,k])) != 0
                println("\t le client $k prend $(value(A[i,j,k])) du P$j a partir du magasin $i")
            end
        end
    end
end

Solution obtenue:
	 benefice = 14.0
	 le client 1 prend 1.0 du P1 a partir du magasin 1
	 le client 2 prend 1.0 du P1 a partir du magasin 1
	 le client 2 prend 1.0 du P2 a partir du magasin 1
	 le client 1 prend 1.0 du P1 a partir du magasin 2
	 le client 2 prend 1.0 du P2 a partir du magasin 2
	 le client 2 prend 1.0 du P2 a partir du magasin 3
